# load-digits
test_size = 0.2, random-state = 2021

In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
from sklearn.datasets import load_digits
digits = load_digits()

In [13]:
print(digits.keys())

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])


In [14]:
X = digits.data
y = digits.target

In [7]:
# Feature 데이터
digits.data.shape

(1797, 64)

In [10]:
digits.target.shape

(1797,)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=digits.target,
    test_size=0.2, random_state=2021
)

# 데이터 스케일링 비교해 SVC로 데이터셋 학습해보기
[참고] https://realblack0.github.io/2020/03/29/normalization-standardization-regularization.html

## 1) StandardScaler
각 feature의 평균을 0, 분산을 1로 변경합니다. 모든 특성들이 같은 스케일을 갖게 된다.

In [28]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
svc = SVC()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
svc.fit(X_train_scale, y_train)
print('Scaled test accuracy : %.3f' %(svc.score(X_test_scale,y_test)))

Scaled test accuracy : 0.981


## 2) RobustScaler
모든 특성들이 같은 크기를 갖는다는 점에서 StandardScaler와 비슷하지만, 평균과 분산 대신 median과 quartile을 사용한다. RobustScaler는 이상치에 영향을 받지 않는다.

In [29]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
svc = SVC()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
svc.fit(X_train_scale, y_train)
print('Scaled test accuracy : %.3f' %(svc.score(X_test_scale,y_test)))

Scaled test accuracy : 0.956


## 3) MinMaxScaler
모든 feature가 0과 1사이에 위치하게 만든다. 데이터가 2차원 셋일 경우, 모든 데이터는 x축의 0과 1 사이에, y축의 0과 1사이에 위치하게 된다.

In [36]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
svc = SVC()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
svc.fit(X_train_scale, y_train)
print('Scaled test accuracy : %.3f' %(svc.score(X_test_scale,y_test)))

Scaled test accuracy : 0.986


## 4) Normalizer
StandardScaler, RobustScaler, MinMaxScaler가 각 columns의 통계치를 이용한다면 Normalizer는 row마다 각각 정규화된다. Normalizer는 유클리드 거리가 1이 되도록 데이터를 조정한다. (유클리드 거리는 두 점 사이의 거리를 계산할 때 쓰는 방법, L2 Distance)

In [31]:
from sklearn.preprocessing import Normalizer
scaler = Normalizer()
svc = SVC()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
svc.fit(X_train_scale, y_train)
print('Scaled test accuracy : %.3f' %(svc.score(X_test_scale,y_test)))

Scaled test accuracy : 0.986


## ※ StandardScaler code, RobustScaler code, MinMaxScaler code, Normalizer code로 각각 스케일링 한 후, 서포트 백터 머신으로 학습시켜보니 MinMaxScaler code, Normalizer code가 0.986으로 정확도가 가장 높았다.

위 결과를 통해 나온 MinMaxScaler code, Normalizer code로 알고리즘 종류별 비교를 하겠다.

# Classification(분류) 알고리즘 종류별 비교
일련의 데이터가 포함되는 기존 카테고리들을 학습하고, 이것을 기반으로 컴퓨터는 데이터의 범주를 구분하여 경계를 나누는 것을 학습한다.  따라서 모델에 입력된 새로운 데이터는 해당 점이 어느 곳에 위치하느냐에 따라 가까운 카테고리 혹은 학습된 알고리즘에 의해 분류하게 되다.
### [참고] https://bangu4.tistory.com/99 [방구의 개발냄새]

## 1) KNN(K-nearest neighbor)
실수 데이터의 경우 유클리드 거리 측정 방식을 사용하고, 범주형 혹은 이진 데이터와 같은 유형의 데이터는 해밍 거리 측정 방식을 사용

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [32]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=1) 
knn.fit(X_train_scale,y_train)

print('Scaled test accuracy : %.3f' %(knn.score(X_test_scale,y_test)))

Scaled test accuracy : 0.978


## 2) Decision Tree(의사결정 트리)
가장 단순한 classifier 중 하나로, decision tree와 같은 도구를 활용하여 모델을 그래프로 그리는 매우 단순한 구조로 되어 있다. 이 방식은 root에서부터 적절한 node를 선택하면서 진행하다가 최종 결정을 내리게 되는 model이다.

In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

dt_clf = DecisionTreeClassifier(random_state=2021)
dt_clf.fit(X_train_scale, y_train)

from sklearn.metrics import accuracy_score
pred = dt_clf.predict(X_test_scale)
accuracy_score(y_test, pred)

0.8777777777777778

## 3) Random Forest
Decision tree가 여러개 모여 Forest를 이룬 것이다.

Decision tree보다 작은 Tree가 여러개 모이게 되어, 모든 트리의 결과들을 합하여 더많은 값을 최종결과로 본다.

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

from sklearn.metrics import accuracy_score

rf_clf = RandomForestClassifier(random_state=2021)
rf_clf.fit(X_train_scale,y_train)
pred = rf_clf.predict(X_test_scale)
accuracy_score(y_test,pred)

0.9611111111111111

## 4) SVM (Support Vector Machine)
서포터 벡터 머신은 기본적으로 Decision Boundary라는 직선이 주어진 상태

앞서 스케일링하여 svm으로 데이터셋을 학습시켜봄
MinMaxScaler code, Normalizer code가 가장 높게 나옴

# 서포터 백터 머신을 사용했을 때, 정확도가 가장 높게 나옴.
그러므로 Grid search을 MinMaxScaler code, Normalizer code 스케일링한 svc에 사용하겠다.

# MinMaxScaler로 스케일링한 후, svm 사용해 grid search

In [49]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
svc = SVC()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
svc.fit(X_train_scale, y_train)
best_score = 0

In [43]:
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [53]:
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # 매개변수의 각 조합에 대해 svc 훈련시킴

        clf = SVC(gamma=gamma, C=C)
        clf.fit(X_train_scale,y_train)
        # 테스트 세트로 svc 평가
        y_test_hat = clf.predict(X_test_scale)
        score = accuracy_score(y_test, y_test_hat)
        #점수가 더 높으면 매개변수와 함께 기록
        if score > best_score:
            best_score = score
            best_parameters = {'C':C, 'gamma':gamma}

print("최고 점수: {:.2f}".format(best_score))
print("최적 파라미터:", best_parameters)

최고 점수: 0.98
최적 파라미터: {'C': 100, 'gamma': 0.001}


# Normalizer로 스케일링한 후, svm 사용해 grid search

In [54]:
from sklearn.preprocessing import Normalizer
scaler = Normalizer()
svc = SVC()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
svc.fit(X_train_scale, y_train)
best_score = 0

In [55]:
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # 매개변수의 각 조합에 대해 svc 훈련시킴

        clf = SVC(gamma=gamma, C=C)
        clf.fit(X_train_scale,y_train)
        # 테스트 세트로 svc 평가
        y_test_hat = clf.predict(X_test_scale)
        score = accuracy_score(y_test, y_test_hat)
        #점수가 더 높으면 매개변수와 함께 기록
        if score > best_score:
            best_score = score
            best_parameters = {'C':C, 'gamma':gamma}

print("최고 점수: {:.2f}".format(best_score))
print("최적 파라미터:", best_parameters)

최고 점수: 0.98
최적 파라미터: {'C': 1, 'gamma': 10}
